In [1]:
!pip install vecstack

In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.tree import DecisionTreeClassifier

import pandas, xgboost, numpy, textblob, string
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers, models, optimizers

import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re

In [20]:


df = pd.read_csv('L:/Thesis/Dateset/FakenewsNet.csv', encoding='latin-1')

df = df.astype(dtype={'title': 'string','url': 'string',
                                'date_published': 'string','author_name': 'string',
                                'content': 'string','label': 'string'})

cols = ['countOfPosWord', 'countOfNegWord', 'NumberOfChar','NumberOfWords','NumberOfSentences','AvgCharPerWord','AvgWordPerSent','NumberOfUpCase','NumberOfPunctuatuion','Orgtitle_contentSim','TopicModellingSim','afinn_score','vader_score_compound','FREI','FKGL','ARI','GFI','CLI']
data = pd.read_csv('L:/Thesis/Dateset/FakenewsNet.csv',  usecols = cols)

df['label'].value_counts()

False    130
True     126
Name: label, dtype: Int64

In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
data_scaled = scaler.fit_transform(data)

numerical_data=list()
for row in range(0,len(df)):
  numerical_data.insert(row,data_scaled[row])
df['numerical_features'] = numerical_data

df = df.astype(dtype={'numerical_features': 'string'})

In [22]:
df['text_features']= df['title'] +' '+ df['content'] +' '+ df['author_name'] +' '+ df['date_published']
df['Complete_Data'] = df['text_features']

In [23]:
for row in range(len(df)):
  df['Complete_Data'][row] = re.sub(r"won\'t", "will not", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"can\'t", "can not", df['Complete_Data'][row])

    # general
  df['Complete_Data'][row] = re.sub(r"n\'t", " not", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'re", " are", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'s", " is", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'d", " would", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'ll", " will", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'t", " not", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'ve", " have", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'m", " am", df['Complete_Data'][row])
    
print("DonePartOne")

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

X = df['Complete_Data']
stop_words = set(stopwords.words("english"))

import re
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document= [word for word in document if word not in stop_words]
    document = [ps.stem(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)
    
df['Complete_Data'] = documents
print("DonePartTwo")

df['Complete_Data'] = documents

df['Complete_Data'] = df['Complete_Data'] + df['numerical_features']

for row in range(0,len(df)):
    df['Complete_Data'][row] = re.sub(r'\[',' ',df['Complete_Data'][row])
    df['Complete_Data'][row] = re.sub(r'\]',' ',df['Complete_Data'][row])
    df['Complete_Data'][row] = re.sub(r'\n',' ',df['Complete_Data'][row])
    df['Complete_Data'][row] = re.sub(r'\s+', ' ', df['Complete_Data'][row], flags=re.I)
    
    
print("DonePartThree")


DonePartOne
DonePartTwo
DonePartThree


In [24]:
import textstat
lenght=list()
for row in range(0,len(df)):
  lenght.insert(row , textstat.lexicon_count(df['Complete_Data'][row]))
df['lenght']=lenght

print('min: ', min(df['lenght']))
print('max: ', max(df['lenght']))

print('mean: {:.2f}'.format(np.mean(df['lenght'])))

print('mean + 2 *sigma: {:.2f}'.format(np.mean(df['lenght'])+ 2.0 * np.std(df['lenght'])))


max_features = int(np.mean(df['lenght'])+ 2.0 * np.std(df['lenght']))
print(max_features)

min:  75
max:  3967
mean: 492.35
mean + 2 *sigma: 1428.56
1428


In [25]:

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,4), max_features= max_features, stop_words={'english'})
tfidf_vect_ngram.fit(df['Complete_Data'])
X =  tfidf_vect_ngram.transform(df['Complete_Data'])

X1=X.toarray()

In [26]:
combine_data=list()

for row in range(0,len(df)):
  combine_data.insert(row,np.hstack((X1[row],numerical_data[row])))
df['combine_data'] = combine_data

data = df['combine_data']

In [42]:
data[1]

array([0.06841088, 0.        , 0.        , ..., 0.08650431, 0.15623724,
       0.01882731])

In [27]:
encoder = preprocessing.LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])

y = df['label'].to_numpy()

In [10]:

x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.2)


print(len(x_train), len(x_test))
print(len(y_train), len(y_test))


20612 5153
20612 5153


In [28]:
from sklearn.utils import class_weight

def compute_classweights(target):
    """
    Computes the weights of the target values based on the samples
    :param target: Y-target variable
    :return: dictionary object
    """
    # compute class weights
    class_weights = class_weight.compute_class_weight('balanced',
                                                     np.unique(target),
                                                     target)
    
    # make the class weight list into dictionary
    weights = {}
    
    # enumerate the list
    for index, weight in enumerate(class_weights):
        weights[index] = weight
        
    return weights

# Get the class weights for the target variable
weights = compute_classweights(y)

In [12]:
weights

{0: 0.9901237414495427, 1: 1.0100752705033715}

In [13]:
models = [
    ensemble.RandomForestClassifier(max_depth=25, min_samples_leaf=1, min_samples_split=2, n_estimators=100),
    
    svm.SVC(C=10, gamma=0.1,kernel='rbf', verbose=True),
    
    xgboost.XGBClassifier(use_label_encoder =False, eval_metric='auc',learning_rate=0.1, 
                  objective='multi:softprob', num_class=5, max_depth= 20, n_estimators=1000),

    linear_model.LogisticRegression(solver='lbfgs', max_iter=100),
    
    DecisionTreeClassifier(criterion='gini', min_samples_split=2, min_samples_leaf=1, max_depth=10, splitter='best')
    
]

In [14]:
from vecstack import stacking
import warnings
warnings.filterwarnings('ignore')

S_train, S_test = stacking(models,                     # list of models
                           x_train.tolist(), y_train, x_test.tolist(),   # data
                           regression=False,           # classification task (if you need 
                                                       #     regression - set to True)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and vote
                           needs_proba=False,          # predict class labels (if you need 
                                                       #     probabilities - set to True) 
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=accuracy_score,      # metric: callable
                           n_folds=4,                  # number of folds
                           stratified=True,            # stratified split for folds
                           shuffle=True,               # shuffle the data
                           random_state=0,             # ensure reproducibility
                           verbose=2)   

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [RandomForestClassifier]
    fold  0:  [0.91791190]
    fold  1:  [0.91150786]
    fold  2:  [0.90859693]
    fold  3:  [0.91131380]
    ----
    MEAN:     [0.91233262] + [0.00342060]
    FULL:     [0.91233262]

model  1:     [SVC]
[LibSVM]    fold  0:  [0.89908791]
[LibSVM]    fold  1:  [0.89869979]
[LibSVM]    fold  2:  [0.88957889]
[LibSVM]    fold  3:  [0.89831166]
    ----
    MEAN:     [0.89641956] + [0.00395899]
    FULL:     [0.89641956]

model  2:     [XGBClassifier]
    fold  0:  [0.92780904]
    fold  1:  [0.93440714]
    fold  2:  [0.92315156]
    fold  3:  [0.93246652]
    ----
    MEAN:     [0.92945857] + [0.00435991]
    FULL:     [0.92945857]

model  3:     [LogisticRegression]
    fold  0:  [0.85600621]
    fold  1:  [0.85658840]
    fold  2:  [0.84552688]
    fold  3:  [0.85328935]
    ----
    MEAN:     [0.85285271] + [0.004409

In [45]:
type(S_test)

numpy.ndarray

In [44]:
model = xgboost.XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                      n_estimators=100, max_depth=3)
    
# Fit 2nd level model
model = model.fit(S_train, y_train)

# Predict
y_pred = model.predict(S_test)

# Final prediction score
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

[21:07:43] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Final prediction score: [0.93033185]


In [17]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=encoder.classes_))

              precision    recall  f1-score   support

       False       0.92      0.94      0.93      2650
        True       0.94      0.92      0.93      2503

    accuracy                           0.93      5153
   macro avg       0.93      0.93      0.93      5153
weighted avg       0.93      0.93      0.93      5153



In [18]:
from sklearn.metrics import confusion_matrix
confusionmatrix = confusion_matrix(y_test,y_pred)
print(confusionmatrix)

[[2503  147]
 [ 212 2291]]


In [19]:
import pickle
file_name = "xgb_stacking.pkl"

# save
pickle.dump(model, open(file_name, "wb"))

In [47]:
test = data.to_numpy()


In [48]:
type(test)

numpy.ndarray

In [49]:
y_pred = model.predict(test)

# Final prediction score
print('Final prediction score: [%.8f]' % accuracy_score(y, y_pred))

XGBoostError: [21:08:40] d:\bld\xgboost-split_1619725139497\work\src\data\array_interface.h:139: Check failed: typestr.size() == 3 (2 vs. 3) : `typestr' should be of format <endian><type><size of type in bytes>.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=encoder.classes_))